# 1)Converti COCO JSON in CSV

In [1]:
import json
import pandas as pd
from pathlib import Path

# Modifica questi percorsi in base alla tua struttura
json_path = Path("/PFE_DETECTION_DATASET.v2i.coco/_annotations.coco.json")
images_folder = Path("/Users/lorenzodimaio/PyCharmMiscProject/PFE_DETECTION_DATASET.v2i.coco/train")

with open(json_path) as f:
    coco = json.load(f)

# Mappa image_id -> file_name
id_to_filename = {img["id"]: img["file_name"] for img in coco["images"]}

# Estrai annotazioni
rows = []
for ann in coco["annotations"]:
    x, y, w, h = ann["bbox"]
    x1, y1 = int(x), int(y)
    x2, y2 = int(x + w), int(y + h)
    filename = id_to_filename[ann["image_id"]]
    rows.append({
        "filename": filename,
        "x1": x1, "y1": y1,
        "x2": x2, "y2": y2,
        "class": ann.get("category_id", 0)
    })

df = pd.DataFrame(rows)
df.to_csv("annotations.csv", index=False)
print("✅ Creato file annotations.csv")


✅ Creato file annotations.csv


# 1) Unificare etichette duplicati con maiuscole/minuscole diverse

In [4]:
import json
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch

class CleanedCOCODataset(Dataset):
    def __init__(self, annotation_file, img_dir, transform=None):
        with open(annotation_file) as f:
            coco_data = json.load(f)

        self.img_dir = Path(img_dir)
        self.transform = transform

        # Step 1: Mappa nomi originali → nome pulito
        name_map = {
            "MOUTH": "mouth",
            "mouth": "mouth",
            "EYE": "eye",
            "eye": "eye",
            "NOSE": "nose"
        }

        # Step 2: Crea nuove categorie pulite e mappa id originale → nuovo id
        category_id_to_name = {cat["id"]: cat["name"] for cat in coco_data["categories"]}
        clean_name_to_id = {}
        old_to_new_category_id = {}

        next_id = 0
        for old_id, name in category_id_to_name.items():
            if name in name_map:
                clean_name = name_map[name]
                if clean_name not in clean_name_to_id:
                    clean_name_to_id[clean_name] = next_id
                    next_id += 1
                old_to_new_category_id[old_id] = clean_name_to_id[clean_name]

        self.label_map = clean_name_to_id  # salva se ti serve la label per ogni ID

        # Step 3: Carica immagini
        self.images = {img["id"]: img for img in coco_data["images"]}

        # Step 4: Riorganizza annotazioni filtrando HEAD, BABY, ecc.
        self.annotations = {}
        for ann in coco_data["annotations"]:
            old_cat_id = ann["category_id"]
            if old_cat_id not in old_to_new_category_id:
                continue  # ignora categorie non mappate (es. HEAD, BABY)

            new_cat_id = old_to_new_category_id[old_cat_id]
            img_id = ann["image_id"]
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            new_ann = {
                "bbox": ann["bbox"],
                "category_id": new_cat_id
            }
            self.annotations[img_id].append(new_ann)

        self.ids = list(self.annotations.keys())

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.images[img_id]
        img_path = self.img_dir / img_info["file_name"]
        image = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        for ann in self.annotations[img_id]:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        if self.transform:
            image = self.transform(image)

        return image, boxes, labels

In [5]:
dataset = CleanedCOCODataset(
    annotation_file="/Users/lorenzodimaio/PyCharmMiscProject/PFE_DETECTION_DATASET.v2i.coco/_annotations.coco.json",
    img_dir="/Users/lorenzodimaio/PyCharmMiscProject/PFE_DETECTION_DATASET.v2i.coco/train",
    #transform=TRANSFORM_FUNZIONE_OPZIONALE
)

In [10]:
dataset.__getitem__(4) #image, boxes, labels = dataset[i]
#'eye': 0, 'mouth': 1, 'nose': 2 -> guarda la labels!!!

(<PIL.Image.Image image mode=RGB size=640x640>,
 tensor([[338.0000, 256.0000, 440.5000, 297.0000],
         [153.0000, 198.0000, 251.0000, 246.5000],
         [203.0000, 272.0000, 328.5000, 344.5000]]),
 tensor([0, 0, 2]))